In [72]:
# Step 2: New Food Supplier Distribution Model
# Using unmet demand from Step 1 to prioritize new supply allocation

import pandas as pd
import numpy as np
import geopandas as gpd
from haversine import haversine, Unit
import gurobipy as gp
from gurobipy import GRB

# Model parameters
PURCHASE_COST_PER_LB = 0.55  # $/lb
TRANSPORT_COST_PER_LB_MILE = 0.10  # $/lb-mile (adjust as needed)
NEW_SUPPLY_AMOUNT = 40_000_000  # 40 million lbs
SUPPLIER_COORDS = (40.8128, -73.8801)  # (lat, lon) - New York distribution center
UNMET_PENALTY_WEIGHT = 0.01  # Weight for population-based unmet demand penalty ($/person-lb)

print("STEP 2: NEW FOOD SUPPLIER OPTIMIZATION MODEL")
print(f"\nModel Parameters:")
print(f"  Purchase cost: ${PURCHASE_COST_PER_LB}/lb")
print(f"  Transport cost: ${TRANSPORT_COST_PER_LB_MILE}/lb-mile")
print(f"  Unmet penalty weight: ${UNMET_PENALTY_WEIGHT}/person-lb")
print(f"  New supply available: {NEW_SUPPLY_AMOUNT:,} lbs")
print(f"  Supplier location: {SUPPLIER_COORDS[0]}°N, {SUPPLIER_COORDS[1]}°W")

STEP 2: NEW FOOD SUPPLIER OPTIMIZATION MODEL

Model Parameters:
  Purchase cost: $0.55/lb
  Transport cost: $0.1/lb-mile
  Unmet penalty weight: $0.01/person-lb
  New supply available: 40,000,000 lbs
  Supplier location: 40.8128°N, -73.8801°W


In [73]:
# Load Step 1 results - pantries with unmet demand
step1_flows = pd.read_csv("data/model/optimal_flows_radius.csv")

# Load data
nta_table = pd.read_csv("data/model/nta_table.csv")
pantries_table = pd.read_csv("data/model/pantries_table.csv")
tracts = pd.read_csv("data/model/tracts_table.csv")

# Clean supply gap column
SUPPLY_GAP_COL = "Supply Gap (lbs.)"
nta_table[SUPPLY_GAP_COL] = nta_table[SUPPLY_GAP_COL].apply(
    lambda x: float(str(x).replace(',', ''))
)

print(f"\nLoaded {len(nta_table)} neighborhoods and {len(pantries_table)} pantries")

# Identify deficit NTAs
DEFICIT_NTA = nta_table[nta_table[SUPPLY_GAP_COL] > 0].copy()
deficit_nta_ids = set(DEFICIT_NTA['nta2020'].astype(str).str.strip().str.upper())

print(f"Deficit NTAs: {len(deficit_nta_ids)}")

# Prepare pantries data
pantries_table['nta2020_clean'] = pantries_table['nta2020'].astype(str).str.strip().str.upper()
nta_table['nta2020_clean'] =  nta_table['nta2020'].astype(str).str.strip().str.upper()
pantries_table["geoid"] = pantries_table["geoid"].astype(str)
tracts["geoid"] = tracts["geoid"].astype(str)

def parse_point(geom_str):
    """Extract (lat, lon) from POINT string"""
    if pd.isna(geom_str):
        return None
    try:
        coords = str(geom_str).replace('POINT (', '').replace(')', '').split()
        return (float(coords[1]), float(coords[0]))  # (lat, lon)
    except:
        return None

# Get deficit pantries with coordinates and population
deficit_pantries = (
    pantries_table
    .loc[pantries_table['nta2020_clean'].isin(deficit_nta_ids),
         ['id', 'geoid', 'nta2020_clean', 'geometry']]
    .merge(nta_table[['nta2020_clean', 'TotalPop', 'Food Insecure Population']], on='nta2020_clean', how='left')
)

deficit_pantries['coords'] = deficit_pantries['geometry'].apply(parse_point)
deficit_pantries = deficit_pantries.dropna(subset=['coords'])
deficit_pantries['id'] = deficit_pantries['id'].astype(str)
deficit_pantries.drop(columns=['geometry'], inplace=True)
deficit_pantries["TotalPop"] = deficit_pantries["TotalPop"].fillna(0)
deficit_pantries["Food Insecure Population"] = deficit_pantries["Food Insecure Population"].fillna(0)

print(f"Deficit pantries with valid coords: {len(deficit_pantries)}")


Loaded 197 neighborhoods and 515 pantries
Deficit NTAs: 142
Deficit pantries with valid coords: 283


In [74]:
deficit_pantries

,id,geoid,nta2020_clean,TotalPop,Food Insecure Population,coords
0,0,36005025500,BX0701,43095.0,12045.0525,"(40.862216353818, -73.905672642358)"
1,1,36005005002,BX0901,71213.0,17525.5193,"(40.8264247055377, -73.8785391641132)"
2,2,36047032500,BK0901,42088.0,7937.7968,"(40.6656935813309, -73.9584754469383)"
3,3,36047058000,BK1502,40527.0,6622.1118,"(40.598606630507, -73.955282193588)"
4,5,36081103202,QN1401,38081.0,7517.1894,"(40.6051553276391, -73.7521784537249)"
...,...,...,...,...,...,...
278,509,36061024500,MN1201,83611.0,22391.0258,"(40.835229265893, -73.940571031506)"
279,510,36085001100,SI0101,18039.0,3640.2702,"(40.637659132778, -74.080221249759)"
280,511,36085009602,SI0201,25313.0,3397.0046,"(40.592418061845, -74.091916938355)"
281,513,36047019200,BK1202,84354.0,19207.4058,"(40.629318426656, -73.998923159204)"


In [75]:
# Compute distance from supplier to each deficit pantry
deficit_pantries['distance_to_supplier'] = deficit_pantries['coords'].apply(
    lambda coord: haversine(SUPPLIER_COORDS, coord, unit=Unit.MILES)
)

print(f"\nDistance to supplier range: {deficit_pantries['distance_to_supplier'].min():.2f} to {deficit_pantries['distance_to_supplier'].max():.2f} miles")


Distance to supplier range: 0.94 to 27.46 miles


In [76]:
# Create NTA lookup dictionary
nta_lookup = {}
for _, row in nta_table.iterrows():
    nta_code = str(row['nta2020']).strip().upper()
    nta_lookup[nta_code] = {
        'gap': float(row[SUPPLY_GAP_COL]),
        'pantry_count': int(row['pantry_count']) if pd.notna(row['pantry_count']) else 1
    }

In [77]:
# Calculate original demand per pantry (before Step 1)
original_demand_per_pantry = {}

for _, row in deficit_pantries.iterrows():
    pantry_id = str(row['id'])
    nta_code = row['nta2020_clean']

    if nta_code in nta_lookup:
        gap = nta_lookup[nta_code]['gap']
        n_pantries = max(nta_lookup[nta_code]['pantry_count'], 1)
        demand_amount = gap / n_pantries if gap > 0 else 0.0
    else:
        demand_amount = 0.0

    original_demand_per_pantry[pantry_id] = demand_amount

In [78]:
# Calculate how much each deficit pantry RECEIVED in Step 1
received_in_step1 = {}
step1_flows['to_pantry'] = step1_flows['to_pantry'].astype(str)
for pantry_id in deficit_pantries['id']:
    received = step1_flows[step1_flows['to_pantry'] == pantry_id]['lbs'].sum()
    received_in_step1[pantry_id] = received

In [79]:
# Calculate remaining unmet demand (what Step 2 needs to address)
unmet_demand_per_pantry = {}

for pantry_id in deficit_pantries['id'].tolist():
    original = original_demand_per_pantry.get(pantry_id, 0.0)
    received = received_in_step1.get(pantry_id, 0.0)
    unmet = max(0.0, original - received)  # Ensure non-negative
    
    unmet_demand_per_pantry[pantry_id] = {
        "original_demand": original,
        "received_step1": received,
        "unmet_demand": unmet
    }

In [80]:
# Build distance and population weight dictionaries
deficit_ids = deficit_pantries['id'].tolist()
dist_to_supplier = {str(row['id']): row['distance_to_supplier'] 
                    for _, row in deficit_pantries.iterrows()}
w_unmet = {str(row["id"]): float(row["Food Insecure Population"])
           for _, row in deficit_pantries.iterrows()}

total_original_demand = sum(d["original_demand"] for d in unmet_demand_per_pantry.values())
total_received_step1 = sum(d["received_step1"] for d in unmet_demand_per_pantry.values())
total_unmet = sum(d["unmet_demand"] for d in unmet_demand_per_pantry.values())

In [81]:
print(f"\n DEMAND ANALYSIS:")
print(f"  Total original demand: {total_original_demand:,.2f} lbs")
print(f"  Total received in Step 1: {total_received_step1:,.2f} lbs ({total_received_step1/total_original_demand*100:.1f}%)")
print(f"  Total unmet demand: {total_unmet:,.2f} lbs ({total_unmet/total_original_demand*100:.1f}%)")
print(f"  New supply available: {NEW_SUPPLY_AMOUNT:,.2f} lbs")
print(f"  Coverage ratio: {NEW_SUPPLY_AMOUNT/total_unmet*100:.1f}%")

# Filter to only pantries with unmet demand > 0
pantries_with_unmet = [pid for pid in deficit_ids if unmet_demand_per_pantry[pid]["unmet_demand"] > 1e-6]
print(f"\n  Pantries with unmet demand: {len(pantries_with_unmet)} out of {len(deficit_ids)}")


 DEMAND ANALYSIS:
  Total original demand: 96,655,913.95 lbs
  Total received in Step 1: 50,589,455.30 lbs (52.3%)
  Total unmet demand: 46,066,458.65 lbs (47.7%)
  New supply available: 40,000,000.00 lbs
  Coverage ratio: 86.8%

  Pantries with unmet demand: 150 out of 283


In [82]:
# BUILD OPTIMIZATION MODEL
print("BUILDING OPTIMIZATION MODEL")

model = gp.Model("new_supplier_distribution")
model.setParam("OutputFlag", 1)

# Decision variables - only for pantries with unmet demand
# y[j] = amount of new food supplied to pantry j
y = model.addVars(pantries_with_unmet, name="y", lb=0.0)

# u[j] = remaining unmet demand at pantry j (after Step 2)
u = model.addVars(pantries_with_unmet, name="u", lb=0.0)

model.update()

print(f"\nVariables created:")
print(f"  Supply variables (y): {len(y):,}")
print(f"  Unmet demand variables (u): {len(u):,}")

# Objective function
# Total cost = purchase cost + transport cost + weighted unmet demand penalty
purchase_cost = PURCHASE_COST_PER_LB * gp.quicksum(y[j] for j in pantries_with_unmet)

transport_cost = gp.quicksum(
    TRANSPORT_COST_PER_LB_MILE * dist_to_supplier[j] * y[j]
    for j in pantries_with_unmet
)

# Weight unmet demand by population to prioritize high-population areas
# Apply penalty weight to scale appropriately with dollar costs
unmet_penalty = UNMET_PENALTY_WEIGHT * gp.quicksum(w_unmet[j] * u[j] for j in pantries_with_unmet)

model.setObjective(purchase_cost + transport_cost + unmet_penalty, GRB.MINIMIZE)

# Constraints
# 1. Demand satisfaction at each pantry (using UNMET demand from Step 1)
for j in pantries_with_unmet:
    model.addConstr(
        y[j] + u[j] == unmet_demand_per_pantry[j]["unmet_demand"],
        name=f"demand_{j}"
    )

# 2. Total supply limit from new supplier
model.addConstr(
    gp.quicksum(y[j] for j in pantries_with_unmet) <= NEW_SUPPLY_AMOUNT,
    name="total_supply_limit"
)

BUILDING OPTIMIZATION MODEL
Set parameter OutputFlag to value 1

Variables created:
  Supply variables (y): 150
  Unmet demand variables (u): 150


<gurobi.Constr *Awaiting Model Update*>

In [83]:
# SOLVE
print("\n" + "="*60)
print("SOLVING OPTIMIZATION MODEL")
print("="*60 + "\n")

model.optimize()


SOLVING OPTIMIZATION MODEL

Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (mac64[rosetta2] - Darwin 24.2.0 24C2101)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 151 rows, 300 columns and 450 nonzeros (Min)
Model fingerprint: 0x178cdfb6
Model has 298 linear objective coefficients
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 4e+07]
Presolve removed 150 rows and 154 columns
Presolve time: 0.00s
Presolved: 1 rows, 146 columns, 146 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0125178e+07   6.608538e+05   0.000000e+00      0s
       1    2.1477581e+08   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.147758093e+08


In [84]:
if model.status == GRB.OPTIMAL:
    print("OPTIMAL SOLUTION FOUND")
    
    # Extract solution
    distributions = []
    for j in pantries_with_unmet:
        supplied = y[j].X
        if supplied > 1e-6:
            distributions.append({
                "pantry_id": j,
                "original_demand": unmet_demand_per_pantry[j]["original_demand"],
                "received_step1": unmet_demand_per_pantry[j]["received_step1"],
                "unmet_before_step2": unmet_demand_per_pantry[j]["unmet_demand"],
                "supplied_step2": supplied,
                "remaining_unmet": u[j].X,
                "distance_miles": dist_to_supplier[j],
                "transport_cost": TRANSPORT_COST_PER_LB_MILE * dist_to_supplier[j] * supplied,
                "purchase_cost": PURCHASE_COST_PER_LB * supplied,
                "total_cost": (PURCHASE_COST_PER_LB + TRANSPORT_COST_PER_LB_MILE * dist_to_supplier[j]) * supplied,
                "food insecure population": w_unmet[j],
            })
    
    distributions_df = pd.DataFrame(distributions)
    
    # Calculate final unmet demand across all pantries
    final_unmet = {j: u[j].X for j in pantries_with_unmet}
    total_final_unmet = sum(final_unmet.values())
    total_unmet_penalty = UNMET_PENALTY_WEIGHT * sum(w_unmet[j] * final_unmet[j] for j in pantries_with_unmet)
    
    # Calculate statistics
    total_supplied = distributions_df['supplied_step2'].sum() if len(distributions_df) > 0 else 0
    total_purchase_cost = distributions_df['purchase_cost'].sum() if len(distributions_df) > 0 else 0
    total_transport_cost = distributions_df['transport_cost'].sum() if len(distributions_df) > 0 else 0
    total_actual_cost = total_purchase_cost + total_transport_cost
    avg_distance = (distributions_df['supplied_step2'] * distributions_df['distance_miles']).sum() / total_supplied if total_supplied > 0 else 0
    
    print(f"\n SOLUTION STATISTICS:")
    print(f"  Pantries supplied in Step 2: {len(distributions)}")
    print(f"  Total food supplied in Step 2: {total_supplied:,.2f} lbs ({total_supplied/NEW_SUPPLY_AMOUNT*100:.1f}% of available)")
    print(f"  Unmet demand addressed: {total_supplied:,.2f} lbs ({total_supplied/total_unmet*100:.1f}% of Step 1 unmet)")
    print(f"  Final remaining unmet demand: {total_final_unmet:,.2f} lbs ({total_final_unmet/total_original_demand*100:.1f}% of original)")
    
    print(f"\n COMPARISON TO STEP 1:")
    print(f"  Original total demand: {total_original_demand:,.2f} lbs")
    print(f"  After Step 1 (unmet): {total_unmet:,.2f} lbs ({total_unmet/total_original_demand*100:.1f}%)")
    print(f"  After Step 2 (unmet): {total_final_unmet:,.2f} lbs ({total_final_unmet/total_original_demand*100:.1f}%)")
    print(f"  Total demand met (Steps 1+2): {(total_received_step1 + total_supplied):,.2f} lbs ({(total_received_step1 + total_supplied)/total_original_demand*100:.1f}%)")
    
    print(f"\n STEP 2 COST BREAKDOWN:")
    print(f"  Purchase cost: ${total_purchase_cost:,.2f}")
    print(f"  Transport cost: ${total_transport_cost:,.2f}")
    print(f"  Unmet demand penalty: ${total_unmet_penalty:,.2f}")
    print(f"  Total actual cost (purchase + transport): ${total_actual_cost:,.2f}")
    print(f"  Average cost per lb delivered: ${total_actual_cost/total_supplied:.3f}" if total_supplied > 0 else "  Average cost per lb: N/A")
    
    print(f"\n DISTANCE STATISTICS:")
    print(f"  Weighted avg distance: {avg_distance:.2f} miles")
    print(f"  Total lb-miles: {(distributions_df['supplied_step2'] * distributions_df['distance_miles']).sum():,.2f}" if len(distributions_df) > 0 else "  Total lb-miles: 0")
    
    print(f"\n OBJECTIVE VALUE BREAKDOWN:")
    print(f"  Purchase + Transport: ${total_actual_cost:,.2f}")
    print(f"  Unmet Penalty: ${total_unmet_penalty:,.2f}")
    print(f"  Total Objective: ${model.objVal:,.2f}")


elif model.status == GRB.INFEASIBLE:
    print("\n MODEL IS INFEASIBLE")
    print("Computing IIS (Irreducible Inconsistent Subsystem)...")
    model.computeIIS()
    model.write("model_step2_iis.ilp")
    print("IIS written to model_step2_iis.ilp")
    
elif model.status == GRB.UNBOUNDED:
    print("\n MODEL IS UNBOUNDED")
    
else:
    print(f"\n Optimization failed with status code: {model.status}")
    print("See Gurobi documentation for status code meanings")

print("STEP 2 MODEL COMPLETE")

OPTIMAL SOLUTION FOUND

 SOLUTION STATISTICS:
  Pantries supplied in Step 2: 126
  Total food supplied in Step 2: 40,000,000.00 lbs (100.0% of available)
  Unmet demand addressed: 40,000,000.00 lbs (86.8% of Step 1 unmet)
  Final remaining unmet demand: 6,066,458.65 lbs (6.3% of original)

 COMPARISON TO STEP 1:
  Original total demand: 96,655,913.95 lbs
  After Step 1 (unmet): 46,066,458.65 lbs (47.7%)
  After Step 2 (unmet): 6,066,458.65 lbs (6.3%)
  Total demand met (Steps 1+2): 90,589,455.30 lbs (93.7%)

 STEP 2 COST BREAKDOWN:
  Purchase cost: $22,000,000.00
  Transport cost: $27,729,927.54
  Unmet demand penalty: $165,045,881.72
  Total actual cost (purchase + transport): $49,729,927.54
  Average cost per lb delivered: $1.243

 DISTANCE STATISTICS:
  Weighted avg distance: 6.93 miles
  Total lb-miles: 277,299,275.43

 OBJECTIVE VALUE BREAKDOWN:
  Purchase + Transport: $49,729,927.54
  Unmet Penalty: $165,045,881.72
  Total Objective: $214,775,809.27
STEP 2 MODEL COMPLETE


In [85]:
# Save results
if len(distributions_df) > 0:
    distributions_df = distributions_df.sort_values('supplied_step2', ascending=False)
    distributions_df.to_csv("data/model/step2_supplier_distributions_food_insecure_pop.csv", index=False)
    print(f"\n Results saved to data/model/step2_supplier_distributions.csv")
    
    print("\n Top 10 distributions by volume:")
    display_cols = ['pantry_id', 'unmet_before_step2', 'supplied_step2', 'remaining_unmet', 'distance_miles', 'total_cost', 'food insecure population']
    print(distributions_df[display_cols].head(10).to_string(index=False))

# Show pantries with highest remaining unmet demand
if total_final_unmet > 0:
    remaining_unmet_df = distributions_df[distributions_df['remaining_unmet'] > 1].copy()
    remaining_unmet_df = remaining_unmet_df.sort_values("remaining_unmet", ascending=False)
    
    print(f"\n  Top 10 pantries with REMAINING unmet demand after Step 2:")
    display_cols_unmet = ['pantry_id', 'original_demand', 'received_step1', 'supplied_step2', 'remaining_unmet', 'food insecure population']
    print(remaining_unmet_df[display_cols_unmet].head(10).to_string(index=False))


 Results saved to data/model/step2_supplier_distributions.csv

 Top 10 distributions by volume:
pantry_id  unmet_before_step2  supplied_step2  remaining_unmet  distance_miles   total_cost  food insecure population
      214        1.804792e+06    1.804792e+06              0.0       15.134479 3.724094e+06                 8776.6242
      416        1.444845e+06    1.444845e+06              0.0        4.568424 1.454731e+06                 7946.5563
      485        1.379134e+06    1.379134e+06              0.0        3.348239 1.220291e+06                 8017.2834
      503        1.208993e+06    1.208993e+06              0.0        6.634057 1.466999e+06                 5468.2056
       33        1.153882e+06    1.153882e+06              0.0        6.975515 1.439527e+06                 4002.8394
      221        1.090395e+06    1.090395e+06              0.0        3.032355 9.303637e+05                 6493.4805
      513        1.088923e+06    1.088923e+06              0.0       14.12203